In [1]:
!pip install --upgrade pip

In [2]:
import sys
sys.version

'3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]'

In [3]:
!pip install JPype1-1.1.2-cp38-cp38-win_amd64.whl

ERROR: JPype1-1.1.2-cp38-cp38-win_amd64.whl is not a supported wheel on this platform.


In [4]:
!pip install konlpy

In [5]:
# 한글 형태소 분류기
from konlpy.tag import Okt, Kkma

In [6]:
okt = Okt()
# 형태소 별로 분류
okt.morphs('아버지가방에들어가신다')

['아버지', '가방', '에', '들어가신다']

In [7]:
kkma = Kkma()
kkma.morphs('아버지가방에들어가신다')

['아버지', '가방', '에', '들어가', '시', 'ㄴ다']

In [8]:
text = '아버지가방에들어가신다'
# 어떤 형태소인지 분류해서 대상을 알려줌
okt.pos(text)

[('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가신다', 'Verb')]

In [9]:
kkma.pos(text)

[('아버지', 'NNG'),
 ('가방', 'NNG'),
 ('에', 'JKM'),
 ('들어가', 'VV'),
 ('시', 'EPH'),
 ('ㄴ다', 'EFN')]

In [10]:
# 분류할 수 있는 형태소를 나열
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [11]:
# 명사만 추출
okt.nouns(text)

['아버지', '가방']

In [12]:
!pip install sklearn numpy pandas matplotlib

In [13]:
# countvectorizer와 연결해서 사용하기 (BOW)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [14]:
text = [ '천혜향 맛있어요',
       '저는 어제 25 21을 봤습니다.',
       '삼겹살 맛있어요']

In [15]:
cv.fit(text)

CountVectorizer()

In [16]:
# 띄어쓰기 단위로 토큰화
cv.vocabulary_

{'천혜향': 7, '맛있어요': 2, '저는': 6, '어제': 5, '25': 1, '21을': 0, '봤습니다': 3, '삼겹살': 4}

In [17]:
# 토큰화 방법 정의 함수
def mytoken(text) :
    return okt.nouns(text)

In [18]:
cv_okt = CountVectorizer( tokenizer = mytoken)

In [19]:
cv_okt.fit(text)

C:\Users\smhrd\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function mytoken at 0x00000269C3E71160>)

In [20]:
cv_okt.vocabulary_

{'천혜향': 3, '저': 2, '어제': 1, '삼겹살': 0}

In [21]:
import pandas as pd

# 문제 정의
- 한글 영화 리뷰 데이터셋으로 감성분석을 진행
- kolnpy를 사용해서 형태소 분류

# 데이터수집

In [24]:
text_train = pd.read_csv('./data/ratings_train.txt' , delimiter = '\t')
text_test = pd.read_csv('./data/ratings_test.txt' , delimiter = '\t')

In [25]:
text_train.shape

(150000, 3)

In [26]:
text_test.shape

(50000, 3)

In [27]:
text_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 데이터 전처리

In [28]:
text_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


## 결측치 삭제

In [29]:
# 결측치가 존재하는 행을 삭제
text_train.dropna(inplace = True)
text_test.dropna(inplace = True)

In [30]:
text_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [32]:
X_train = text_train['document']
y_train = text_train['label']
X_test = text_test['document']
y_test = text_test['label']

## 토큰화

In [33]:
# 토큰화 방법 정의 함수
def mytoken(text) :
    return okt.nouns(text)

In [34]:
cv_okt = CountVectorizer( tokenizer = mytoken )

In [35]:
cv_okt.fit(X_train)

C:\Users\smhrd\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function mytoken at 0x00000269C53BDC10>)

In [36]:
len(cv_okt.vocabulary_)

38648

In [37]:
# 수치화
X_train_okt = cv_okt.transform(X_train)
X_test_okt = cv_okt.transform(X_test)